# This notebook will pretrain a BERT model on the WikiText-2 dataset

## Let's first load the WikiText-2 dataset

In [1]:
#let's get the data..let's try to use the Hugging Face API

from datasets import load_dataset

# Load the wikitext-2-raw-v1 configuration of the wikitext dataset
raw_datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

# The 'raw_datasets' object will now contain the dataset, typically split into
# 'train', 'validation', and 'test' splits.
# You can access them like this:
train_data = raw_datasets["train"]
validation_data = raw_datasets["validation"]
test_data = raw_datasets["test"]

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(validation_data)}")
print(f"Number of test examples: {len(test_data)}")

Number of training examples: 36718
Number of validation examples: 3760
Number of test examples: 4358


## Let's create a custom Dataset and Collate function for dynamic padding

In [2]:
import random
from transformers import AutoTokenizer, DataCollatorForLanguageModeling
import torch

checkpoint = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, data, max_length):
        self.tokenizer=tokenizer
        self.max_length = max_length
        
        L = [p["text"] for p in data]
        self.NSP_data = []

        for paragraph in L:
            cand_sentences = paragraph.split(' . ')
            number_of_sentences = len(cand_sentences)
            if number_of_sentences<2:
                continue
            else:
                for i in range(number_of_sentences-1):
                    if random.random()<0.5:
                        self.NSP_data.append(((cand_sentences[i], cand_sentences[i+1]), 1))
                else:
                    repla_sent = random.choice(L)
                    repla_sent = repla_sent.split(' . ')
                    repla_sent = random.choice(repla_sent)
                    self.NSP_data.append(((cand_sentences[i], repla_sent), 0))


    
    def __getitem__(self, idx):
        pair, label = self.NSP_data[idx]
        return self.tokenizer(*pair, truncation=True, max_length=self.max_length), label
        
    def __len__(self):
        return len(self.NSP_data)


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

def collate_fn(batch):
    tokens = [item[0] for item in batch]
    
    tokens = data_collator(tokens)  #output will be {"input_ids" : ? , "token_type_ids" : ?, "attention_mask" : ?, "labels" : ?}


    mask = (tokens["labels"] == -100)

    mlm_pred_positions = tokens["labels"].clone()
    mlm_pred_positions[mask] = 0
    mlm_pred_positions_batch, mlm_pred_positions_pos = torch.nonzero(mlm_pred_positions, as_tuple=True)
    mlm_preds = torch.nonzero(mlm_pred_positions)
    tokens["labels"] = tokens["labels"][mlm_preds[:,0], mlm_preds[:,1]]
    

    #swap values 1<->0 in attention_mask as nn.TransformerEncoderLayer uses a different convention
    mask = (tokens["attention_mask"]==1)
    tokens["attention_mask"][mask] = 0
    tokens["attention_mask"][~mask] = 1
    tokens["attention_mask"] = tokens["attention_mask"].bool()
            
    nsp_labels = [item[1] for item in batch]
    
    nsp_labels = torch.LongTensor(nsp_labels)
    X = {k:v for k, v in tokens.items() if k != "labels"}  #X excludes "labels", as this concerns more the output.
    X["labels_batch"] = mlm_pred_positions_batch
    X["labels_positions"] = mlm_pred_positions_pos
    y = (tokens["labels"], nsp_labels)
    #y = (tokens["labels"], torch.zeros(len(batch), dtype=torch.int64))
    return X,y

2025-09-01 05:06:53.958370: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 05:06:53.966594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756696013.976669 1731760 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756696013.979628 1731760 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-01 05:06:53.990078: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Let's now build the BERT model and custom criterion used

In [3]:
import torch.nn as nn
import math

#custom model
class BertPretraining(nn.Module):
    def __init__(self, vocab_size, num_hiddens, ffn_num_hiddens, num_heads, num_blks, dropout, max_len):
        super().__init__()
        self.num_hiddens=num_hiddens
        self.max_len = max_len
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)
        self.segment_embedding = nn.Embedding(2, num_hiddens)
        self.pos_embedding = nn.Parameter(torch.randn(1,max_len, num_hiddens))
        self.blks = nn.Sequential()
        for i in range(num_blks):
            self.blks.add_module(f"{i}", nn.TransformerEncoderLayer(d_model=num_hiddens, 
                                                                   nhead=num_heads,
                                                                   dim_feedforward=ffn_num_hiddens,
                                                                   dropout=dropout, 
                                                                   batch_first=True))
            
        self.mlp = nn.Sequential(nn.LazyLinear(num_hiddens), nn.ReLU(), nn.LayerNorm(num_hiddens), nn.LazyLinear(vocab_size))

        self.nsp = nn.Sequential(nn.LazyLinear(num_hiddens), nn.Tanh(), nn.LazyLinear(2))

    
            

    def forward(self, input_ids, token_type_ids, attention_mask, labels_batch, labels_positions):
        X = self.token_embedding(input_ids)*math.sqrt(self.num_hiddens) + self.segment_embedding(token_type_ids)
        X = X + self.pos_embedding[:, :X.shape[1], :]
        for blk in self.blks:
            X = blk(X, src_key_padding_mask=attention_mask)

        ##get mlm_Y_hat prediction
        masked_X = X[labels_batch, labels_positions]
        mlm_Y_hat = self.mlp(masked_X)

        #get nsp_Y_hat prediction
        nsp_Y_hat  =self.nsp(X[:,0,:])

        return mlm_Y_hat, nsp_Y_hat

    
    def apply_init(self, inputs, init=None):
        self.forward(**inputs)
        if init is not None:
            self.mlp.apply(init)
            self.nsp.apply(init)
            self.blks.apply(init)

    def init_weights_xavier_uniform(self, m):
        if hasattr(m, 'weight') and m.weight.dim()>=2:
            nn.init.xavier_uniform_(m.weight)


#custom criterion
class CriterionBert(torch.nn.modules.loss._Loss):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, y_pred, y_true):
        loss = nn.CrossEntropyLoss()
        
        #compute masked language model loss:
        mlm_l = loss(y_pred[0], y_true[0].reshape(-1))

        #compute next sentence prediction loss:
        nsp_l = loss(y_pred[1], y_true[1].reshape(-1))

        #total loss
        l = mlm_l + nsp_l

        return l

## Optuna based objective function for hyperparameter tuning

In [4]:
#approach using optuna

import optuna
from sklearn.model_selection import KFold
import gc





def objective(trial):

    #define hyperparameter search space. Keep it simple and just search "lr" and "weight_decay"
    
    lr = trial.suggest_float("lr", 1e-5, 1e0, log=True)
    
    weight_decay = trial.suggest_float("Ridge", 1e-5, 1e0, log=True)

    #define model, optimizer and criterion    
    model = BertPretraining(vocab_size = len(tokenizer), num_hiddens=128, ffn_num_hiddens=256, num_heads=2, num_blks=2, 
                            dropout=0.2, max_len=15)
    device = torch.device("cuda:0")
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    criterion = CriterionBert()


    #define data and cross-validation splitting values

    dataset = MyDataset(tokenizer, train_data, 15)

    kf = KFold(n_splits=3, shuffle=True)
    fold_accuracies=[]

    for train_idx, val_idx in kf.split(range(len(dataset))):
        
        train_subset = torch.utils.data.Subset(dataset, train_idx)
        val_subset = torch.utils.data.Subset(dataset, val_idx)

        train_dataloader = torch.utils.data.DataLoader(train_subset, batch_size=1024, shuffle=True, pin_memory=True, num_workers=10, 
                                                       collate_fn = collate_fn, prefetch_factor=2, multiprocessing_context='fork')

        val_dataloader = torch.utils.data.DataLoader(val_subset, batch_size=1024, shuffle=False, pin_memory=True, num_workers=10, 
                                                    collate_fn = collate_fn, prefetch_factor=2, multiprocessing_context='fork')
        
        #re-initialize weights here
        init_inputs = (next(iter(train_dataloader)))[0]
        for k, v in init_inputs.items():
            v = v.to(device, non_blocking=True)
            init_inputs[k]=v
            
        model.apply_init(init_inputs, model.init_weights_xavier_uniform)

        num_epochs=5
        for epoch in range(num_epochs):
            model.train()
            for inputs, labels in train_dataloader:
                #move data to GPU
                for k,v in inputs.items():
                    v = v.to(device, non_blocking=True)
                    inputs[k] = v

                labels_mlm = labels[0].to(device, non_blocking=True)
                labels_nsp = labels[1].to(device, non_blocking=True)
                labels = (labels_mlm, labels_nsp)
                
                outputs = model(**inputs)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                del inputs
                del labels
                del loss
                del outputs

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_dataloader:

                for k, v in inputs.items():
                    v = v.to(device, non_blocking=True)
                    inputs[k] = v

                labels_mlm = labels[0].to(device, non_blocking=True)
                labels_nsp = labels[1].to(device, non_blocking=True)
                labels = (labels_mlm, labels_nsp)
                
                total += labels[0].shape[0] + labels[1].shape[0]
                outputs = model(**inputs) #mlm_Y_hat and nsp_Y_hat
                #get predictions from mlm_Y_hat
                _, mlm_preds = torch.max(outputs[0],1)
                correct += (mlm_preds==labels[0]).sum().item()

                #get predictions from nsp_Y_hat
                _, nsp_preds = torch.max(outputs[1],1)
                correct += (nsp_preds==labels[1]).sum().item()

                del inputs
                del labels
                del outputs
                del mlm_preds
                del nsp_preds

        fold_accuracies.append(correct/total)
    del model
    torch.cuda.empty_cache()
    gc.collect()

    return sum(fold_accuracies)/len(fold_accuracies)

## Run the optuna study

In [5]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=3, n_jobs=3, show_progress_bar=True)
torch.cuda.empty_cache()  #clear reserved memory

[I 2025-09-01 05:07:06,205] A new study created in memory with name: no-name-50396b93-ec55-45dd-b0d6-8286f8c6c045


  0%|          | 0/3 [00:00<?, ?it/s]

[I 2025-09-01 05:09:32,307] Trial 0 finished with value: 0.30118768338327717 and parameters: {'lr': 0.0010611174682805564, 'Ridge': 0.09270599575803558}. Best is trial 0 with value: 0.30118768338327717.
[I 2025-09-01 05:09:32,325] Trial 2 finished with value: 0.2810787355447523 and parameters: {'lr': 0.005973141263366834, 'Ridge': 0.00019930307397072425}. Best is trial 0 with value: 0.30118768338327717.
[I 2025-09-01 05:09:35,154] Trial 1 finished with value: 0.27845738822513183 and parameters: {'lr': 0.046420052131932385, 'Ridge': 0.02903435311088126}. Best is trial 0 with value: 0.30118768338327717.


## Appendix

### In this appendix we play around with the dataset to get a feel for its output signature

#### The dataset uses the tokenizer to create the __getitem__. Let's see what it looks like

In [580]:
L = [p["text"] for p in train_data]

import random
NSP_data = []  #elements will be (sen1, sen2, True/False) depending on whether sen2 follows sen1

for paragraph in L:
    cand_sentences = paragraph.split(' . ')
    number_of_sent = len(cand_sentences)
    if number_of_sent<2:
        continue
    else:
        for i in range(number_of_sent-1):
            if random.random()<0.5:
                NSP_data.append(((cand_sentences[i], cand_sentences[i+1]), 1))
            #look at cand_sentences[i] and cand_sentences[i+1]
            else:
                repla_sent = random.choice(L)
                repla_sent = repla_sent.split(' . ')
                repla_sent = random.choice(repla_sent)
                NSP_data.append(((cand_sentences[i], repla_sent), 0))

In [585]:
pair, _ = NSP_data[5]
tokenizer(*pair, truncation=True, max_length=70)

{'input_ids': [101, 1799, 1122, 5366, 1103, 2530, 1956, 1104, 1103, 1326, 117, 1122, 1145, 9315, 2967, 27939, 117, 1216, 1112, 1543, 1103, 1342, 1167, 1111, 5389, 3970, 1111, 1326, 25551, 1116, 102, 134, 134, 22130, 134, 134, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### Let's see how collate function pads/batches data together 

In [588]:
pair1, _ = NSP_data[5]
pair2, _ = NSP_data[6]

x = tokenizer(*pair1, truncation=True, max_length=70)
y = tokenizer(*pair2, truncation=True, max_length=70)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)

tokens = data_collator([x,y])
tokens

{'input_ids': tensor([[  101,   103,  1122,  5366,  1103,  2530,  1956,  1104,  1103,  1326,
           117,  1122,  1145,  9315,  2967, 27939,   117,  1216,  1112,  1543,
          1103,  1342,  1167,  1111,   103,   103,   103,  1326, 25551,  1116,
           102,   134,   134, 22130,   134,   103,   102,     0,     0,     0,
             0,     0],
        [  101, 23543,  5592,   103,  1777, 10942, 25028,  1105,  3996, 15375,
           103, 17784, 18504, 12355,  1241,  1608,  1121,  2166, 10813,   117,
          1373,  1114,   103,   103,  3464, 17758,  1563,  1900, 26713, 18763,
         16075, 10946,   102,   138,   103,  1264,  1104,  5094,  8630,  1103,
          5444,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
dataset = MyDataset(tokenizer, train_data, 15)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=3, collate_fn=collate_fn)

X, y = next(iter(dataloader))
print("X:", X,"\n","y:", y)

X: {'input_ids': tensor([[  101, 11930,   103,  1356,  1349,   103,  1999,   102, 18653,  1643,
           103,  1158,   103,  1269,   102],
        [  101, 18653,  1643, 26179,   103,  1103,  1269, 11970,  1104, 12394,
          1105,   103,   137,   102,   102],
        [  101, 18653,  1643, 26179,   103,  1103,  1269,   102,   134,  3506,
          7085, 19975,  1105, 19415,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False]]), 'labels_batch': tensor([0, 0, 0,